 #             Import  required library  

In [25]:
import json
import sys
import os
import time

# DataStore Creating and CRD Implementation 

In [26]:
# define Python user-defined exceptions
class Error(Exception):
    pass


class keySizeException(Error):
    pass
class valueSizeException(Error):
    pass
class fileSizeException(Error):
    pass
class keyDuplicate(Error):
    pass
class emptyFileException(Error):
    pass
class keyNotFound(Error):
    pass
class expiredException(Error):
    pass

class datastore:
    FILE_NAME = ""
    def __init__(self,*args):
        try:

            if(len(args) == 0):
                nm = "default.json"
            
            else:
                nm = args[0]
           
            f = open(nm,"x")
            datastore.FILE_NAME = nm
            print("file created ")
        except:
            print("file already exists")
    
        
    def createdata(self,key,value,*args):
        #key length should be <=32   
        try:
            if(len(key) > 32):
                raise keySizeException
          
        except keySizeException:
            print("Key length is restricted to 32 chracters. Kindly shorten your key to 32 characters or less.\n")
           
            
        #value size <=16kB   
        try:
            valueSize = sys.getsizeof(value)/1024
            if(valueSize > 16):
                raise valueSizeException
        except valueSizeException:
            print("JSON value must be less than 16kB\n")
         
            
        #exsisting filsesize<=1gB
        try:
            existingFileSize = os.path.getsize(datastore.FILE_NAME)
            existingFileSize = (existingFileSize/1024)/1024 #MB
            
            if(existingFileSize >= 1024):
                raise fileSizeException
               
            if(existingFileSize == 0.0):#empty file
                timeToLive = -1
                if(len(args) != 0):#timetolive val passed
                    timeToLive = args[0]
                created = int(time.time())
                try:
                    #writeToFile(key,value,timeToLive)
                    json_data = {key : {"value":value,
                               "timeToLive":timeToLive,
                                "created":created
                               }}
                    #print(json_data)
                    with open(datastore.FILE_NAME, 'w') as json_file:
                        json.dump(json_data, json_file)
                        print("Record Created\n")

                except:
                    print("IO Exception\n")
            else:
                #data present in json file
                
                #load file data
                with open(datastore.FILE_NAME) as f:
                    data = json.load(f)
                try:
                    if(key in set(data.keys())):
                        raise keyDuplicate
                            
                    timeToLive = -1
                    if(len(args) != 0):#timetolive val passed
                        timeToLive = args[0]
                    created = int(time.time())
                    try:
                        json_data = {key : {"value":value,
                               "timeToLive":timeToLive,
                                "created":created
                               }}
                        #print(json_data)
                        data.update(json_data)
                        with open(datastore.FILE_NAME, 'w') as json_file:
                            json.dump(data, json_file)
                            print("Record Created\n")

                    
                    except:
                        print("IO Exception\n") 
                        
                    
                except keyDuplicate:
                    print("Key already Exists\n")
                    
        except fileSizeException:
            print("File size exceeded")
            print()
            
    def readData(self,key):
        try:
            existingFileSize = os.path.getsize(datastore.FILE_NAME)   
            if(existingFileSize == 0.0):
                raise emptyFileException
            with open(datastore.FILE_NAME) as f:
                data = json.load(f)
            try:
                s = set(data.keys())
                if(key not in s):
                    raise keyNotFound
                        
                created = data[key]['created']
                timeToLive = data[key]['timeToLive']
                if(timeToLive == -1):
                    return data[key]['value']
                else:
                    try:
                        curDate = int(time.time())
                        if(curDate >= created + timeToLive):
                            raise expiredException
                        return data[key]['value']
                    except expiredException:
                        print("Key Exceeded Time To Live")
                
            except keyNotFound:
                print("Key Not Found\n")
                    
                    
        except emptyFileException:
            print("Datastore is Empty\n")
                
    def delData(self,key):
        try:
            existingFileSize = os.path.getsize(datastore.FILE_NAME)   
            if(existingFileSize == 0.0):
                raise emptyFileException
            with open(datastore.FILE_NAME) as f:
                data = json.load(f)
            try:
                s = set(data.keys())
                if(key not in s):
                    raise keyNotFound
                        
                created = data[key]['created']
                timeToLive = data[key]['timeToLive']
                if(timeToLive == -1):
                    del data[key]
                    with open(datastore.FILE_NAME, 'w') as json_file:
                        json.dump(data, json_file)
                    print("Record Deleted\n")
                else:
                    try:
                        curDate = int(time.time())
                        if(curDate >= created + timeToLive):
                            raise expiredException
                        
                        del data[key]
                        with open(datastore.FILE_NAME, 'w') as json_file:
                            json.dump(data, json_file)
                        print("Record Deleted\n")
                        
                    except expiredException:
                        print("Key Exceeded Time To Live")
                
            except keyNotFound:
                print("Key Not Found\n")
                    
                    
        except emptyFileException:
            print("Datastore is Empty\n")
                
                        

# Testing CRD operation with sample inputs 

## Checking for file creation 

In [27]:
# creating file without giveing  file name 
d = datastore()

file created 


In [28]:
# file   created with parameter 
d = datastore("data.json")

file created 


In [29]:
# try to create exiting file 
d = datastore("data.json")

file already exists


## Create Record

In [30]:
# record created
x = {
  "name": "John",
  "age": 30,
  "city": "New York"
}

y = json.dumps(x)
d.createdata("first",y)



Record Created



In [31]:
# try to create existing record 
x = {
  "name": "John",
  "age": 30,
  "city": "New York"
}

y = json.dumps(x)
d.createdata("first",y)

Key already Exists



In [32]:
# try to create record with timeToLive 
x = {
  "name": "John",
  "age": 30,
  "city": "New York"
}

y = json.dumps(x)
d.createdata("second",y,60)#RECORD EXPIRES IN 60 SECONDS


Record Created



## Read Data

In [33]:
# read existing key  
d.readData("first")

'{"name": "John", "age": 30, "city": "New York"}'

In [34]:
# try to read non existing key 
d.readData("third")

Key Not Found



In [37]:
# try to read expire record   
d.readData("second")

Key Exceeded Time To Live


## Delete data 

In [38]:
# delete existing data
d.delData("first")

Record Deleted



In [39]:
# delete non  existing data
d.delData("third")

Key Not Found



In [40]:
# delete expired data 
d.delData("second")

Key Exceeded Time To Live
